# 4camping_workspace_ keboola

Tady sepišeme krok za krokem, jaké transformace jsme udělali v keboolu abysme ziskali naše končné data k vizualizace
- prvni pokusi před Hackatonem: 
    -  vybirat a stahovat tabulky z firemniho database
    - vytvorit datový model
    - nahrat data do keboolo
- Hackaton:
    - spojime zbytecne zmatujici a komplikovane tabulky, hodne veci jeste nevime jak funguje v keboolo co taky neurychlí prace
    - nejakou tabulku jsme potom dali do Tableau, tady taky jsme meli problem, jak updatovat, pokud nejaky data nasledně měnime ve keboolo
    - zkusili jsme přidat kraj, okres, psc prodejny, pomoci regex udelat slopuce kde jsou barvy

Vysledek Hackatonu, že začnem znova spolu vytvořit naše zakladnou tabulku, v tomhle workbooku budem sepsat kroky


Extra info:
- pojmenovano tabulky ve storage (CAPS_LOCK)
- tabulky z DB nahrany pres STORAGE
- tabulky ohledne county, region nahrany pres COMPONENT/GOOGLE DRIVE 

- vice o vztazich v datovém modelu
https://lucid.app/lucidchart/5b471e84-a440-4587-a1cf-19c93dbd04c6/edit?invitationId=inv_0c0c0ddc-16c1-424f-ae3e-6f41d12bd1f1&page=wbRCCwA8fE17#

#### _1_4CAMPING_TRANSPORTS_ZIP

POJENY TABULKY RŮZNÉ SHIPPING METHODS K ZISKAVANI GEOUDAJU K PICKUP_POINTUM A BRANCH
INUPUT : 13 dopravni tabulka
OUTPUT: TRANSPORT

In [ ]:
-- pomoci unionem vytvorim tabulku kde budeme mit PSC od vsech dopravce podle orders_id

ALTER TABLE "sm_balik_napostu"    --balik na postu nemel long a lat
ADD COLUMN latitude VARCHAR(90);

ALTER TABLE "sm_balik_napostu"
ADD COLUMN longitude VARCHAR(90);

CREATE OR REPLACE TABLE TRANSPORTS AS
select
 zo."orders_id" AS T_ORDERS_ID
,z."zip" AS T_ZIP
,z."street" AS T_STREET
,z."city" AS T_CITY
,z."latitude" AS T_LAT
,z."longitude" AS T_LONG
from "sm_zasilkovna_branches" as z
Left Join "sm_zasilkovna_orders" as zo ON zo."branches_id" = z."id"
where z."country" = 'cz'

UNION ALL

select
 bo."orders_id" AS T_ORDERS_ID
,b."zipcode"  AS T_ZIP
,SPLIT_PART(b."address", ',',1) AS T_STREET --tady je jeste problem, mame celou adresu v poli street
,b."city" AS T_CITY
,b."latitude" AS T_LAT
,b."longitude" AS T_LONG
from "sm_balikovna_branches" as b
Left Join "sm_balikovna_orders" as bo ON bo."branches_id" = b."id"

UNION ALL

select
dpdo."orders_id" AS T_ORDERS_ID
,dpd."postcode"   AS T_ZIP
,dpd."street_address" AS T_STREET
,dpd."city" AS T_CITY
,dpd."latitude" AS T_LAT
,dpd."longitude" AS T_LONG
From "sm_dpd_parcel_shops_branches" as dpd
Left Join "sm_dpd_parcel_shop_orders" as dpdo ON dpdo."branches_id" = dpd."id"

UNION ALL

select
go."orders_id" AS T_ORDERS_ID
,g."zipcode"  AS T_ZIP
,g."street" AS T_STREET
,g."city" AS T_CITY
,g."gpsn" AS T_LAT
,g."gpse" AS T_LONG
from "sm_geis_point_branches" as g
Left Join "sm_geis_orders" as go ON go."branches_id" = g."id"
where g."country" = 'CZ'

UNION ALL

select
  b."orders_id" AS T_ORDERS_ID
  ,bo."zipcode"  AS T_ZIP
  ,SPLIT_PART(bo."address", ',',1) AS T_STREET
  ,bo."township" AS T_CITY
  ,bnp."Latitude" AS T_LAT --doplneno
  ,bnp."Longitude" AS T_LONG --doplneno
from "sm_balik_napostu" b
LEFT JOIN  "sm_post_oofices" bo ON b."zipcode" = bo."zipcode"
LEFT JOIN "balik_na_postu_zip" bnp ON bnp."Zip_Code" = b."zipcode" --doplneno

UNION ALL

select
"orders_id" T_ORDERS_ID
,"zip"  T_ZIP
,"name" T_STREET
,"town" T_CITY
,"latitude"  T_LAT
,"longitude"  T_LONG
from ulozenka
where "country" = 'CZE'

#### _2_4CAMPING_ORDER_GEO

VYTVORENI TABULKU GEO UDAJEMA,KTERÝ PAK MUZEME NAPOJIT S TABULKOU ORDERS_ITEMS POMOCI (VYTVORIME VE 2 KROCICH)
INUPUT : 8 tabulka
OUTPUT: ORDER_GEO
        ORDERS_PSC (pomocná)



In [ ]:
# 1_block:

CREATE OR REPLACE TABLE BALIKOBOT AS
select *
from balikobot
where "rec_country" = 'CZ';

UPDATE ORDERS
SET "date" = DATE(TO_TIMESTAMP("date", 'YYYY-MM-DD HH24:MI:SS'));

# 2_block

--NEJDRIV OPRAVIT CHYBNY UDAJ V DATABAZI - GEIS POINT - OSOBNI ODBER JE TYPE BRANCH, MUSIME OPRAVIT NA PICKUP_POINT
UPDATE SHIPPING_METHODS
SET "type" = 'pickup_point'
WHERE "id" = '79';

--Spojime tabulku orders transportem, balikobotem, a zipcodestore-em

CREATE OR REPLACE TABLE ORDERS_PSC AS
select 
 O."id" AS ID
,O."date" AS DATE_O
,O."number" AS ORDER_NUMBER
,O."street" AS STREER_FROM_ORDERS
,O."delivery_street" AS DELIVERY_STREET_ORDERS
,B."rec_street"AS STREET_BALIKOBOT
,SPLIT_PART(S.STORE_ADDRESS,',',1) AS STORE_ADDRESS  
,T.T_STREET AS STREET_TRANSPORT
,O."zipcode" AS ZIP_FROM_ORDER
,B."rec_zip" AS ZIP_BALIKOBOT
,O."city" AS CITY_FROM_ORDER
,B."rec_city" AS CITY_FROM_BALIKOBOT
,O."delivery_zipcode" AS DELIVERY_ZIP_ORDER
,O."delivery_city" AS DELIVERY_CITY_ORDER
,S.STORE_ZIP_CODE AS ZIP_CODE
,T.T_ZIP AS ZIP_TRANSPORT
,T.T_CITY AS CITY_TRANSPORT
,S.STORE_LAT AS S_LAT
,T.T_LAT
,T.T_LONG
,S.STORE_LONG AS S_LONG
,S.STORE_CITY AS S_CITY --pridala jsem do store - city - pridavam sem
,O."shipping_methods_name_cs" AS SHIPPING_METHOD
,O."shipping_methods_id"  AS SHIPPING_METHOD_ID
,SM."type" AS TYPE_DELIVERY
,SPLIT ("date", '-')[0] AS IYEAR
from orders O
LEFT JOIN STORE_TABLEAU AS S ON S.id_shipping_method = O."shipping_methods_id"
LEFT JOIN transports AS T ON T.t_orders_id = O."id"
LEFT JOIN balikobot as B ON O."id" = B."order_id"
LEFT JOIN shipping_methods as SM on O."shipping_methods_id" = SM."id"
where "countries_id" = '1'               ---country jenom cz
AND iyear between '2017' AND '2022' ;    --- roky jenom mezi 2022-2017

# 3_block

DELETE FROM ORDERS_PSC
WHERE SHIPPING_METHOD_ID IN                   
('4'
,'78'
,'89'
,'102'
,'107'
,'108'
,'109'
,'114'
,'117'
,'133'
,'137'
,'139'
,'140'
,'157'
,'158'
,'169'
,'172'
,'174'
,'178'
,'181'
,'183'
,'186'
,'190'
,'187') ;

DELETE FROM ORDERS_PSC
WHERE ZIP_CODE IN (SELECT "psc" FROM "psc_sk");

DELETE FROM ORDERS_PSC
WHERE ZIP_CODE IN (
'Praha4'
,'Nová885'
, 'G427RN'
, '7019900'
,'12345'
,'00100');


UPDATE ORDERS_PSC
SET ZIP_CODE = ZIP_TRANSPORT
WHERE ZIP_CODE IS NULL  or ZIP_CODE = '' ;

UPDATE ORDERS_PSC
SET ZIP_CODE =  ZIP_BALIKOBOT
WHERE ZIP_CODE IS NULL  or ZIP_CODE = '' ;

UPDATE ORDERS_PSC
SET ZIP_CODE =  DELIVERY_ZIP_ORDER
WHERE ZIP_CODE IS NULL  or ZIP_CODE = '' ;

UPDATE ORDERS_PSC
SET ZIP_CODE =  DELIVERY_ZIP_ORDER
WHERE ZIP_CODE IS NULL  or ZIP_CODE = '' ;

UPDATE ORDERS_PSC
SET ZIP_CODE =  ZIP_FROM_ORDER
WHERE ZIP_CODE IS NULL  or ZIP_CODE = '' ;

UPDATE ORDERS_PSC  --pridala jsem po pridani sloupecku s_city do transports
SET CITY_TRANSPORT =  S_CITY
WHERE CITY_TRANSPORT IS NULL  or CITY_TRANSPORT = '' ;

UPDATE ORDERS_PSC  --pridala jsem po pridani sloupecku s_city do transports
SET  S_CITY = CITY_TRANSPORT 
WHERE S_CITY IS NULL or S_CITY = '' ;

UPDATE ORDERS_PSC
SET S_CITY =  CITY_FROM_BALIKOBOT
WHERE S_CITY IS NULL  or S_CITY = '' ;

UPDATE ORDERS_PSC
SET S_CITY =  DELIVERY_CITY_ORDER
WHERE S_CITY IS NULL  or S_CITY = '' ;

UPDATE ORDERS_PSC
SET S_CITY =  CITY_FROM_ORDER
WHERE S_CITY IS NULL  or S_CITY = '' ;

UPDATE ORDERS_PSC
SET  STORE_ADDRESS = STREET_TRANSPORT
WHERE STORE_ADDRESS IS NULL  or STORE_ADDRESS = '' ;

UPDATE ORDERS_PSC
SET STORE_ADDRESS =  STREET_BALIKOBOT
WHERE STORE_ADDRESS IS NULL  or STORE_ADDRESS = '' ;

UPDATE ORDERS_PSC
SET STORE_ADDRESS =  DELIVERY_CITY_ORDER
WHERE STORE_ADDRESS IS NULL  or STORE_ADDRESS = '' ;

UPDATE ORDERS_PSC
SET STORE_ADDRESS =   STREER_FROM_ORDERS
WHERE STORE_ADDRESS IS NULL  or STORE_ADDRESS = '' ;

UPDATE ORDERS_PSC
SET S_LONG =  T_LONG
WHERE S_LONG IS NULL  or S_LONG = '' ;

UPDATE ORDERS_PSC
SET S_LAT =  T_LAT
WHERE S_LAT IS NULL  or S_LAT = '' ;

# 4_block

CREATE OR REPLACE TABLE ORDERS_GEO AS
select
id
,STORE_ADDRESS||', '||S_CITY ||', '||ZIP_CODE AS address
,UPPER(TRANSLATE(CONCAT(STORE_ADDRESS,S_CITY,ZIP_CODE), 'áčďéěíňóřšťúůýžÁČĎÉĚÍŇÓŘŠŤÚŮÝŽ', 'acdeeinorstuuyzACDEEINORSTUUYZ')) AS ID_CONCAT
,STORE_ADDRESS AS STREET
,S_CITY AS CITY
--,UPPER(TRANSLATE(STORE_ADDRESS, 'áčďéěíňóřšťúůýžÁČĎÉĚÍŇÓŘŠŤÚŮÝŽ', 'acdeeinorstuuyzACDEEINORSTUUYZ')) AS STREET  -- varianta s velkyma pismenama a bez diakritik
--,UPPER(TRANSLATE(S_CITY, 'áčďéěíňóřšťúůýžÁČĎÉĚÍŇÓŘŠŤÚŮÝŽ', 'acdeeinorstuuyzACDEEINORSTUUYZ')) AS CITY
,zip_code
,s_lat AS LAT
,s_long AS LONG
,order_number
,shipping_method
,type_delivery
from orders_psc

#### PYTHON_1

PRES PYTHON JSME ZÍSKALI : Latitude, Longitude, SubRegion, Region, Municipality, Neighborhood, PostalCode
Po vice variantech, na druche Hacketonu jsem dostali klíč od 


In [ ]:
import boto3
import json
import os.path
import csv

places = ["Komenského 100, Holice, 53401"]
cache = {}

if os.path.isfile('location-cache.json'):
    with open('location-cache.json', 'r', encoding="utf-8") as file:
        cache = json.loads(file.read())

session = boto3.session.Session(
    aws_access_key_id='XXXXXXXXXXXXX',
    aws_secret_access_key='XXXXXXXXXXXXXXXXXX'
)
location_client = session.client(service_name='location', region_name='eu-west-1')

num = 0
with open('dta_complete.csv', 'w', encoding='utf-8', newline='') as output_file:
    writer = csv.writer(output_file)
    writer.writerow(["Address", "Latitude", "Longitude", "SubRegion", "Region", "Municipality", "Neighborhood", "PostalCode"])
    with open('dta_missing_1.csv', encoding='utf-8') as input_file:
        for line in input_file:
            place = line.strip().strip('"')
            num = num + 1
            if num == 1:
                continue
            if num % 100 == 0:
                print(num)
                print(json.dumps(cache, default=str), file=open('location-cache.json', 'w', encoding="utf-8"))
            if num > 225000:
                break
            if place not in cache:
                try:
                    response = location_client.search_place_index_for_text(
                        IndexName='MyPlaceIndex',
                        Language='cs',
                        MaxResults=1,
                        Text=place
                    )
                    cache[place] = response['Results'][0]['Place']
                except IndexError:
                    # Handle the case when response['Results'] is empty or IndexError occurs
                    print(f"No results found for place: {place}")
                    continue

            neighborhood = cache[place].get("Neighborhood", "")
            postalCode = cache[place].get("PostalCode", "")
            subregion = cache[place].get("SubRegion", "")
            latitude = cache[place]["Geometry"]["Point"][1]
            longitude = cache[place]["Geometry"]["Point"][0]
            region = cache[place].get("Region", "")
            municipality = cache[place].get("Municipality", "")

            radek = [place, latitude, longitude, subregion, region, municipality, neighborhood, postalCode]
            writer.writerow(radek)

print(json.dumps(cache, default=str), file=open('location-cache.json', 'w', encoding="utf-8"))

#### _2B_4CAMPING_ORDER_GEO_DTA_MISSING

VYTVORENI TABULKU GEO UDAJEMA. 2 KROK. DOPLNIME CHYBEJICI UDAJE, CO JSME ZISKALI PRES PYTHON
INUPUT : 2 tabulka
OUTPUT: GEOLOCATION_TABLEAU

In [ ]:
# 1_block
--kde region nedava smysl - nastavit dohledane hodnoty na N/A

update python_union_all
set "PostalCode" = 'n/a'
where "Region" not in (
'Středočeský kraj'
,'Hlavní město Praha'
,'Jihomoravský kraj'
,'Moravskoslezský kraj'
,'Ústecký kraj'
,'Jihočeský kraj'
,'Olomoucký kraj'
,'Královéhradecký kraj'
,'Zlínský kraj'
,'Pardubický kraj'
,'Plzeňský kraj'
,'Liberecký kraj'
,'Kraj Vysočina'
,'Karlovarský kraj'
,'Vysočina'
,'Jihomoravský'
,'Jihočeský'
,'Moravskoslezský'
,'Praha'
,'Zlínský'
,'Ústecký'
,'Olomoucký'
,'Královéhradecký'
,'Pardubický'
,'Jihočeský'
,'Liberecký'
,'Karlovarský'
,'Středočeský'
);

update python_union_all
set "Neighborhood" = 'n/a'
where "Region" not in (
'Středočeský kraj'
,'Hlavní město Praha'
,'Jihomoravský kraj'
,'Moravskoslezský kraj'
,'Ústecký kraj'
,'Jihočeský kraj'
,'Olomoucký kraj'
,'Královéhradecký kraj'
,'Zlínský kraj'
,'Pardubický kraj'
,'Plzeňský kraj'
,'Liberecký kraj'
,'Kraj Vysočina'
,'Karlovarský kraj'
,'Vysočina'
,'Jihomoravský'
,'Jihočeský'
,'Moravskoslezský'
,'Praha'
,'Zlínský'
,'Ústecký'
,'Olomoucký'
,'Královéhradecký'
,'Pardubický'
,'Jihočeský'
,'Liberecký'
,'Karlovarský'
,'Středočeský'
);

update python_union_all
set "Municipality" = 'n/a'
where "Region" not in (
'Středočeský kraj'
,'Hlavní město Praha'
,'Jihomoravský kraj'
,'Moravskoslezský kraj'
,'Ústecký kraj'
,'Jihočeský kraj'
,'Olomoucký kraj'
,'Královéhradecký kraj'
,'Zlínský kraj'
,'Pardubický kraj'
,'Plzeňský kraj'
,'Liberecký kraj'
,'Kraj Vysočina'
,'Karlovarský kraj'
,'Vysočina'
,'Jihomoravský'
,'Jihočeský'
,'Moravskoslezský'
,'Praha'
,'Zlínský'
,'Ústecký'
,'Olomoucký'
,'Královéhradecký'
,'Pardubický'
,'Jihočeský'
,'Liberecký'
,'Karlovarský'
,'Středočeský'
);

update python_union_all
set "SubRegion" = 'n/a'
where "Region" not in (
'Středočeský kraj'
,'Hlavní město Praha'
,'Jihomoravský kraj'
,'Moravskoslezský kraj'
,'Ústecký kraj'
,'Jihočeský kraj'
,'Olomoucký kraj'
,'Královéhradecký kraj'
,'Zlínský kraj'
,'Pardubický kraj'
,'Plzeňský kraj'
,'Liberecký kraj'
,'Kraj Vysočina'
,'Karlovarský kraj'
,'Vysočina'
,'Jihomoravský'
,'Jihočeský'
,'Moravskoslezský'
,'Praha'
,'Zlínský'
,'Ústecký'
,'Olomoucký'
,'Královéhradecký'
,'Pardubický'
,'Jihočeský'
,'Liberecký'
,'Karlovarský'
,'Středočeský'
);

update python_union_all
set "Region" = 'n/a'
where "Region" not in (
'Středočeský kraj'
,'Hlavní město Praha'
,'Jihomoravský kraj'
,'Moravskoslezský kraj'
,'Ústecký kraj'
,'Jihočeský kraj'
,'Olomoucký kraj'
,'Královéhradecký kraj'
,'Zlínský kraj'
,'Pardubický kraj'
,'Plzeňský kraj'
,'Liberecký kraj'
,'Kraj Vysočina'
,'Karlovarský kraj'
,'Vysočina'
,'Jihomoravský'
,'Jihočeský'
,'Moravskoslezský'
,'Praha'
,'Zlínský'
,'Ústecký'
,'Olomoucký'
,'Královéhradecký'
,'Pardubický'
,'Jihočeský'
,'Liberecký'
,'Karlovarský'
,'Středočeský'
);

update python_union_all
set "Latitude" = 'n/a'
where "Region" not in (
'Středočeský kraj'
,'Hlavní město Praha'
,'Jihomoravský kraj'
,'Moravskoslezský kraj'
,'Ústecký kraj'
,'Jihočeský kraj'
,'Olomoucký kraj'
,'Královéhradecký kraj'
,'Zlínský kraj'
,'Pardubický kraj'
,'Plzeňský kraj'
,'Liberecký kraj'
,'Kraj Vysočina'
,'Karlovarský kraj'
,'Vysočina'
,'Jihomoravský'
,'Jihočeský'
,'Moravskoslezský'
,'Praha'
,'Zlínský'
,'Ústecký'
,'Olomoucký'
,'Královéhradecký'
,'Pardubický'
,'Jihočeský'
,'Liberecký'
,'Karlovarský'
,'Středočeský'
);

update python_union_all
set "Longitude" = 'n/a'
where "Region" not in (
'Středočeský kraj'
,'Hlavní město Praha'
,'Jihomoravský kraj'
,'Moravskoslezský kraj'
,'Ústecký kraj'
,'Jihočeský kraj'
,'Olomoucký kraj'
,'Královéhradecký kraj'
,'Zlínský kraj'
,'Pardubický kraj'
,'Plzeňský kraj'
,'Liberecký kraj'
,'Kraj Vysočina'
,'Karlovarský kraj'
,'Vysočina'
,'Jihomoravský'
,'Jihočeský'
,'Moravskoslezský'
,'Praha'
,'Zlínský'
,'Ústecký'
,'Olomoucký'
,'Královéhradecký'
,'Pardubický'
,'Jihočeský'
,'Liberecký'
,'Karlovarský'
,'Středočeský'
);

--vytvorim sloupec REGION, kde budou sjednocene nazvy kraju (u nekterych zaznamu nevyjelo slovo kraj)

update python_union_all
set "Region" =
        case
            when "Region" = 'Jihomoravský' then 'Jihomoravský kraj'
            when "Region" = 'Jihočeský' then 'Jihočeský kraj'
            when "Region" = 'Vysočina' then 'Kraj Vysočina'
            when "Region" = 'Moravskoslezský' then 'Moravskoslezský kraj'
            when "Region" = 'Praha' then 'Hlavní město Praha'
            when "Region" =  'Zlínský' then 'Zlínský kraj'
            when "Region" = 'Ústecký' then 'Ústecký kraj'
            when "Region" = 'Olomoucký' then 'Olomoucký kraj'
            when "Region" = 'Královéhradecký' then 'Královéhradecký kraj'
            when "Region" = 'Pardubický' then 'Pardubický kraj'
            when "Region" = 'Plzeňský' then 'Plzeňský kraj'
            when "Region" = 'Liberecký' then 'Liberecký kraj'
            when "Region" = 'Karlovarský' then 'Karlovarský kraj'
            when "Region" = 'Středočeský' then 'Středočeský kraj'
            else "Region"
        end;

# 2_block

--DOPLNENI SOURADNIC DO ORDERS_GEO Z PYTHONU

CREATE OR REPLACE TABLE MISSING_GEO AS 
select 
g.id
,g.id_concat
,g.address
,mg1."Region" as REGION
,mg1."SubRegion" as SUBREGION
,mg1."Municipality" as CITY
,g.city as g_city
,mg1."Neighborhood" as NEIGHBORHOOD
,mg1."PostalCode" as ZIP_CODE
,g.zip_code as g_zipcode
,mg1."Latitude" as LAT
,g.lat as g_lat
,mg1."Longitude" as LONG
,g.long as g_long
,g.shipping_method
,g.type_delivery
,g.order_number
from orders_geo g
left join python_union_all mg1 on mg1."Address" = g.address;

# 3_block
--UPDATE, PRIPRAVA

UPDATE MISSING_GEO
SET REGION = ''
--SET SUBREGION = ''
WHERE REGION IN (
'Hidalgo'
,'Valga Maakond'
,'San Luis Potosí'
,'راس الخيمة'
,'חיפה' 
,'Rapla Maakond'
,'Ségou'
,'Ida-Viru Maakond'
,'תל אביב'
,'Durango'
,'Nuevo León'
,'أبوظبي'
,'Misiones'
,'Attiki'
,'Guanajuato'
,'מרכז'
,'Puebla'
,'Oaxaca'
,'דרום'
);

UPDATE MISSING_GEO
--SET REGION = ''
SET SUBREGION = ''
WHERE SUBREGION IN (
'Märjamaa'
,'Petah Tikva'
,'Otepää'
,'Tlatlauquitepec'
,'San Bernardo'
,'אשקלון'
,'الظفرة'
,'San Nicolás de los Garza'
,'راس الخيمة'
,'חיפה'
,'Yaonáhuac'
,'San Juan Bautista Tuxtepec'
,'Cárdenas'
,'León'
,'Tulancingo de Bravo'
,'Candelaria'
,'Durango'
,'Tepeyahualco'
,'תל אביב'
,'Chignautla'
,'Hueyapan'
,'Valga'
,'רחובות'
,'Santiago Huauclilla'
,'Villa Hidalgo'
,'Atempan'
,'Lüganuse'
,'Bla'
,'San Francisco del Rincón'
,'Νέα Σμύρνη'
,'Cuyoaco'
,'Topia'
,'العين'
);

UPDATE MISSING_GEO
SET CITY  = G_CITY
WHERE CITY = '';

UPDATE MISSING_GEO
SET ZIP_CODE  = '602 00'
WHERE ZIP_CODE = '';

DELETE FROM MISSING_GEO
WHERE G_ZIPCODE = '';

--4 UPLNE PRAZDNY ZAZNAM

UPDATE MISSING_GEO
SET G_LAT  = LAT
WHERE G_LAT = '';

UPDATE MISSING_GEO
SET G_LONG  = LONG
WHERE G_LONG = '';

#4_bock

CREATE OR REPLACE TABLE GEOLOCATION_TABLEAU AS 

SELECT
ID
,ID_CONCAT
,ADDRESS
,REGION
,SUBREGION
,CITY
,G_CITY
,NEIGHBORHOOD
,ZIP_CODE
,G_ZIPCODE
,G_LAT AS LAT
,G_LONG AS LONG
,TYPE_DELIVERY
FROM MISSING_GEO

#### PYTHON_2

POMOCI GENDER_GESSER JSME ROZKLICOVALI POHLAVI PODLE KRESTNI JMENA
https://pypi.org/project/gender-guesser/

In [ ]:
import pandas as pd
import gender_guesser.detector as gender

# Načtení tabulky z CSV souboru
customer_tableau = pd.read_csv('customers.csv')

# Vytvoření instance detektoru pohlaví
detector = gender.Detector()

# Iterace přes záznamy v tabulce
for index, row in customer_tableau.iterrows():
    first_name = row['name']
    
    # Odhadnutí pohlaví na základě křestního jména
    gender = detector.get_gender(first_name)
    
    # Aktualizace hodnoty pohlaví ve sloupci
    customer_tableau.loc[index, 'pohlavi'] = gender

# Uložení aktualizované tabulky zpět do CSV souboru
customer_tableau.to_csv('customer_gender.csv', index=False)

#### _3_4CAMPING_CUSTOMERS

Z TETO TRANSFORMACE MAME TABULKU ORDER_TABLEAU A CUSTOMER_TABLEAU
INPUT:
OUTPUT: CUSTOMER_TABLEAU
        ORDER_TABLEAU

In [ ]:
# 1_block
---- TUHLE TABULKU NASLEDNE PROZENU PYTHONEM GENDER GUESSEREM A VYTVORIM TABULKU CUSTOMER_GENDER, SE KTEROU BUDU DAL PRACOVAT
--
--create or replace table CUSTOMER_TABLEAU as (
--select
--    "name"
--    ,"surname"
--    ,"email"
--    ,min("date") as first_order
--    ,row_number() over (order by first_order) as ID_CUSTOMER
--from orders
--where 1=1
--	AND "countries_id" = 1
--group by "name", "surname", "email"
--order by first_order)

# 2_block
-- UPDATE GENDER UNKNOWN Z PYTHONU
update CUSTOMER_TABLEAU
set "pohlavi" =
    CASE
        WHEN "surname" ilike '%á' THEN 'female'
        WHEN "surname" ilike '%ova' THEN 'female'
        ELSE 'male'
    END
where "pohlavi" = 'andy';

update CUSTOMER_TABLEAU
set "pohlavi" =
    CASE
        WHEN "surname" ilike '%á' THEN 'female'
        WHEN "surname" ilike '%ova' THEN 'female'
        ELSE 'male'
    END
where "pohlavi" = 'mostly_female';

update CUSTOMER_TABLEAU
set "pohlavi" =
    CASE
        WHEN "surname" ilike '%á' THEN 'female'
        WHEN "surname" ilike '%ova' THEN 'female'
        ELSE 'male'
    END
where "pohlavi" = 'mostly_male';

update CUSTOMER_TABLEAU
set "pohlavi" =
    CASE
        WHEN "surname" ilike '%á' THEN 'female'
        WHEN "surname" ilike '%ova' THEN 'female'
        WHEN "name" ilike '%ová' THEN 'female'
        WHEN "name" ilike '%á' THEN 'female'
        WHEN "name" ilike '%ova' THEN 'female'
        WHEN "name" in ('Marketa'
,'Kristyna'
,'sarka'
,'Eliska'
,'petra'
,'Mariia'
,'lenka'
,'jana'
,'EVA'
,'katka'
,'martina'
,'LUCIE'
,'zuzana'
,'michaela'
,'veronika'
,'katerina'
,'hana'
,'monika'
,'Halyna'
,'Stepanka'
,'zdenka'
,'Snizhana'
,'anna'
,'alena'
,'Alzbeta'
,'tereza') THEN 'female'
        WHEN "name" in ('Jiri'
,'Matěj'
,'jan'
,'petr'
,'Vaclav'
,'martin'
,'pavel'
,'jiří'
,'michal'
,'Šimon'
,'Frantisek'
,'Ales'
,'jaroslav'
,'josef'
,'miroslav'
,'Matyáš'
,'MIlan'
,'tomáš'
,'tomas'
,'jakub'
,'david'
,'KAREL'
,'Vit'
,'Mirek'
,'Matouš'
,'roman'
,'marek'
,'radek'
,'Lubos'
,'milos'
,'jindrich'
,'Ludek'
,'Dusan'
,'ZDENEK'
,'Zbynek'
,'zdeněk'
,'ondřej'
,'Jiři'
,'Honza'
,'daniel'
,'LIBOR'
,'lukas'
,'ladislav'
,'Vojta'
,'filip'
,'ONDREJ'
,'stanislav'
,'Ondra'
,'Oldrich'
,'Jáchym'
,'ROBERT'
,'VÁCLAV'
,'ivo'
,'pan'
,'ALEŠ'
,'vladimír'
,'lukáš'
,'IVAN'
,'Jarda'
,'FRANTIŠEK'
,'kamil'
,'Čeněk'
,'Tomaš'
,'radim'
,'Ctirad'
,'vladimir'
,'adam'
,'Premysl'
,'Vitezslav'
,'Vítek'
,'richard'
,'miloslav'
,'PATRIK'
,'vlastimil'
,'Krystof'
,'viktor'
,'miloš'
,'jindřich'
,'Matus'
,'VOJTĚCH'
,'vojtech'
,'Tobiáš'
,'jaromír'
,'MARCEL'
,'M.'
,'M'
,'Leos'
,'Ing.'
,'Matyas'
,'vít'
,'peter'
,'LUBOŠ'
,'DaN'
,'ANTONIN'
,'LUBOMÍR'
,'jaromir'
,'antonín'
,'ROSTISLAV'
,'MICHAEL'
,'DUŠAN'
,'Hanuš'
,'marian'
,'dalibor'
,'Mojmir'
,'radovan'
,'oldřich'
,'Evzen'
,'Borek'
,'Vašek'
,'vladislav'
,'Slávek'
,'lubomir'
,'rudolf'
,'Lukaš'
,'IGOR'
,'štěpán'
,'stepan'
,'Matous'
,'luděk'
,'bohumil'
,'Tadeas'
,'Mikoláš'
,'Jiří '
,'Mikulas'
,'jozef'
,'BOHUSLAV'
,'Boleslav'
,'Arnost'
,'Yevhen'
,'Vitek'
,'jirka'
,'J.'
,'EDUARD'
,'Bedrich'
,'Bretislav'
,'zbyněk'
,'v.'
,'Tomáš '
,'P.'
,'otakar'
,'Kuba'
,'Jan '
,'Bohumir'
,'Přemek'
,'Petr '
,'Martin '
,'Jirí'
,'Franta'
,'Vitězslav'
,'Věroslav'
,'Ruda'
,'R'
,'matej'
,'Michal '
,'L'
,'JURAJ'
,'HYNEK'
,'Zoltan'
,'Tonda'
,'T'
,'stefan'
,'LEOŠ'
,'jachym'
,'Ing. Petr'
,'Zdeněk '
,'ŠTEFAN'
,'otta'
,'Miloň'
,'Metoděj'
,'Jíří'
,'Ing. Jan'
,'ERIK'
,'ALEXANDR'
,'V'
,'simon'
,'rené'
,'Otomar'
,'Maxmilian'
,'Ludovit'
,'boris'
,'vítězslav'
,'Vavřinec'
,'vilem'
,'rosta'
,'radoslav'
,'přemysl'
,'NORBERT'
,'P'
,'Matin'
,'Jeroným'
,'andrej'
,'alex') THEN 'male'
        ELSE 'unknown'
    END
where "pohlavi" = 'unknown';

# 3_block
--DOPLNENI ID_CUSTOMER DO TABULKY ORDERS_TABLEAU

create or replace table ORDER_TABLEAU AS (
select 
     o."id" AS ID
  	,DATE(TO_TIMESTAMP(o."date", 'YYYY-MM-DD HH24:MI:SS')) as IDATE
    ,cfo.ID_CUSTOMER AS ID_CUSTOMER
    ,o."shipping_methods_id" AS SHIPPING_METHODS_ID
    ,o."payment_terms_name_cs" as PAYMENT_METHODS_NAME
    ,o."total_weight" AS TOTAL_WEIGHT
    ,o."price_czk" AS PRICE_CZK
from orders o
left join CUSTOMER_TABLEAU cfo
on cfo."name" = o."name"
and cfo."surname" = o."surname"
and cfo."email" = o."email"
where o."countries_id" = '1' and idate between '2017-01-01' and '2022-12-31');

#### _4A_4CAMPING_ITEM
VNORENY SELECT UNION PRODUKTU, JOINOVANI S CAT, PROD, ORDERS, SHIPPINGMETHODS

INPUT: 7 tabulka
OUTPUT: ORDER_PRODUCTS_0


In [ ]:
CREATE OR REPLACE TABLE ORDER_PRODUCTS_0 AS ( 

SELECT 
    OP0.*
    ,O."city" AS CITY
    ,O."date" AS DATES
    ,O."shipping_methods_id" 
    ,O."number" AS ORDER_NUMBER --CISLO OBJEDNÁVKY
    ,P."name_cs" AS PRODUCT_NAME
    ,C."name_cs" AS SUBCATEGORIES_NAME
    ,SM."name_cs" AS SHIPPING_METHOD
    ,SPLIT(C."name_full_cs", '||')[0] AS CATEGORIES
FROM 
(SELECT
   OP1."id" AS ID
   ,OP1."date_inserted"
   ,OP1."orders_id" AS ORDERS_ID
   ,OP1."products_id" AS PRODUCTS_ID
   ,OP1."variant_name_cs" AS VARIANT_NAME_CS
   ,OP1."quantity" AS QUANTITY
   ,OP1."price_czk" AS PRICE_CZK
   ,OP1."weight" AS WEIGHT
FROM ORDER_PRODUCT_1 AS OP1

UNION ALL

SELECT
    OP2."id"
    ,OP2."date_inserted"
   ,OP2."orders_id"
   ,OP2."products_id"
   ,OP2."variant_name_cs"
   ,OP2."quantity"
   ,OP2."price_czk"
   ,OP2."weight"
FROM ORDER_PRODUCT_2 as OP2

UNION ALL

SELECT
    OP3."id"
    ,OP3."date_inserted"
   ,OP3."orders_id"
   ,OP3."products_id"
   ,OP3."variant_name_cs"
   ,OP3."quantity"
   ,OP3."price_czk"
   ,OP3."weight"
FROM ORDER_PRODUCT_3 as OP3)  AS OP0

INNER JOIN ORDERS AS O --INNER JOIN JE PROTO, ZE TABULKY ORDERS A PRODUCT_ORDERS BYLY STAZENY V RUZNOU DOBU A NE VSECHNY PO JSOU V O
ON OP0.ORDERS_ID = O."id"
LEFT JOIN PRODUCTS AS P
ON OP0.PRODUCTS_ID = P."id"
LEFT JOIN CATEGORIES AS C
ON C."id" = P."categories_id"
LEFT JOIN SHIPPING_METHODS AS SM
ON SM."id" = O."shipping_methods_id"
)

#### _4B_4CAMPING_ITEM

Z TABULKY ORDER_PRODUCT_0 VYTVORIME ORDER_ITEM, OPRAVIME COLOUR, CHATEGORIE, SIZE
INPUT:
OUTPUT: ORDER_ITEM

In [ ]:
# 1_block
--UPDATE COLOUR

--OBOHACENI TAB ORDER_PRODUCT_0 - BARVY, VELIKOST (!!!! DODELAT), VALUE, UPRAVA DAT.TYPU U PRICE A QNTY
-- PRIDANI SLOUPCE PRO BARVU - ZISKANI BARVY REGEXEM Z VARIANT_NAME_CS - STEJNYM ZPUSBEM TREBA ZISKAT I VELIKOST - POZOR BARVY TREBA DOLADIT
ALTER TABLE ORDER_PRODUCTS_0 
ADD COLUMN PRODUCT_COLOR VARCHAR(50);

UPDATE ORDER_PRODUCTS_0
SET PRODUCT_COLOR = TRIM(REPLACE(regexp_substr(VARIANT_NAME_CS, 'Barva:([^\/]*)'),'Barva: ', ''));

-- U NĚKTERÝCH PRODUKTŮ CHYBÍ NÁZEV (CELKEM JE CCA 360) ZA OBDOBÍ CO PROZKOUMÁME
UPDATE ORDER_PRODUCTS_0
SET PRODUCT_NAME = 'Unknown' 
WHERE PRODUCT_NAME = ''	;

--DO BUDOUCNA PRIDAME SLOUPEC NA VELIKOST + BARVY JE POTREBA DOLADIT (VIZ. 
ALTER TABLE ORDER_PRODUCTS_0 
ADD COLUMN PRODUCT_SIZE VARCHAR(50);

--SELECT PRODUCT_COLOR, COUNT(PRODUCT_COLOR) AS VYSKYT
--FROM ORDER_PRODUCTS_0
--WHERE IYEAR = '2022'
--GROUP BY PRODUCT_COLOR
--ORDER BY VYSKYT DESC)

------------------------------------
--MAME PROBLEM PRAZDNYCH HODNOT VE SLOUPCICH, SE KTERYMI CHCEME POCITAT - ZJISTILY JSME, ZE SE JEDNA O CHYBEJICI HODNOTU U PRODUKTU SLUNECNI BRYLE. DOPOCITALY JSME PRUMERNOU CENU (223), ABYCHOM JI DO PRAZDNYCH HODNOT NAHRADILY.
--SELECT PRODUCTS_ID, AVG(PRICE_CZK) AS PRUMER
--FROM ORDER_PRODUCTS_0
--WHERE PRODUCTS_ID = '18409' AND PRICE_CZK <> ''--
--GROUP BY PRODUCTS_ID

ALTER TABLE ORDER_PRODUCTS_0
ADD COLUMN COLOR VARCHAR(50);

UPDATE ORDER_PRODUCTS_0
SET COLOR = 'ČERNÁ'
WHERE
  UPPER(VARIANT_NAME_CS) ILIKE '%CERN%'
  OR VARIANT_NAME_CS ILIKE '%ČERN%'
  OR UPPER(VARIANT_NAME_CS) ILIKE '%BLAC%'
   OR UPPER(VARIANT_NAME_CS) ILIKE '%BLK%';

UPDATE ORDER_PRODUCTS_0
SET COLOR = 'BÍLÁ'
WHERE
  UPPER(VARIANT_NAME_CS) ILIKE 'BIL%'
  OR VARIANT_NAME_CS ILIKE 'BÍL%'
  OR UPPER(VARIANT_NAME_CS) ILIKE 'WHIT%'
  OR VARIANT_NAME_CS ILIKE '%WHT%';

UPDATE ORDER_PRODUCTS_0
SET COLOR = 'ČERVENÁ'
WHERE
  UPPER(VARIANT_NAME_CS) ILIKE '%CERVE%'
  OR VARIANT_NAME_CS ILIKE '%ČERVE%'
  OR UPPER(VARIANT_NAME_CS) ILIKE '%RED%'
  OR VARIANT_NAME_CS ILIKE '%RUB%';

UPDATE ORDER_PRODUCTS_0
SET COLOR = 'MODRÁ'
WHERE
  UPPER(VARIANT_NAME_CS) ILIKE '%MODR%'
  OR VARIANT_NAME_CS ILIKE '%NAVY%'
  OR UPPER(VARIANT_NAME_CS) ILIKE '%BLUE%'
  OR VARIANT_NAME_CS ILIKE '%BLU%'
  OR VARIANT_NAME_CS ILIKE '%LAGOO%'
  OR VARIANT_NAME_CS ILIKE '%KORÁ%'
  OR VARIANT_NAME_CS ILIKE '%LGN%'
  OR VARIANT_NAME_CS ILIKE '%OCEAN%';

UPDATE ORDER_PRODUCTS_0
SET COLOR = 'ZELENÁ'
WHERE
  UPPER(VARIANT_NAME_CS) ILIKE '%ZELEN%'
  OR VARIANT_NAME_CS ILIKE '%ZELEN%'
  OR UPPER(VARIANT_NAME_CS) ILIKE '%GREEN%'
  OR VARIANT_NAME_CS ILIKE '%LIME%';

UPDATE ORDER_PRODUCTS_0
SET COLOR = 'HNĚDÁ'
WHERE
  UPPER(VARIANT_NAME_CS) ILIKE '%HNED%'
  OR VARIANT_NAME_CS ILIKE '%HNĚD%'
  OR UPPER(VARIANT_NAME_CS) ILIKE '%BROWN%'
  OR VARIANT_NAME_CS ILIKE '%CHOC%';

UPDATE ORDER_PRODUCTS_0
SET COLOR = 'ŽLUTÁ'
WHERE
  UPPER(VARIANT_NAME_CS) ILIKE '%ZLUT%'
  OR VARIANT_NAME_CS ILIKE '%ŽLUT%'
  OR UPPER(VARIANT_NAME_CS) ILIKE '%YEL%'
  OR VARIANT_NAME_CS ILIKE '%LEMON%';

UPDATE ORDER_PRODUCTS_0
SET COLOR = 'ŠEDÁ'
WHERE
  UPPER(VARIANT_NAME_CS) ILIKE '%SED%'
  OR VARIANT_NAME_CS ILIKE '%ŠED%'
  OR UPPER(VARIANT_NAME_CS) ILIKE '%GREY%'
  OR VARIANT_NAME_CS ILIKE '%ANT%'
  OR VARIANT_NAME_CS ILIKE '%GRA%';

UPDATE ORDER_PRODUCTS_0
SET COLOR = 'RŮŽOVÁ'
WHERE
  UPPER(VARIANT_NAME_CS) ILIKE '%RUZ%'
  OR VARIANT_NAME_CS ILIKE '%RŮŽO%'
  OR UPPER(VARIANT_NAME_CS) ILIKE '%PINK%'
  OR VARIANT_NAME_CS ILIKE '%PNK%'
  OR VARIANT_NAME_CS ILIKE '%LOSO%'
  OR VARIANT_NAME_CS ILIKE '%ROSE%';

UPDATE ORDER_PRODUCTS_0
SET COLOR = 'ORANŽOVÁ'
WHERE
  UPPER(VARIANT_NAME_CS) ILIKE '%ORA%'
  OR VARIANT_NAME_CS ILIKE '%CIH%'
  OR UPPER(VARIANT_NAME_CS) ILIKE '%ORANG%'
  OR VARIANT_NAME_CS ILIKE '%ORN%';

UPDATE ORDER_PRODUCTS_0
SET COLOR = 'ARMY'
WHERE
  UPPER(VARIANT_NAME_CS) ILIKE '%MASK%'
  OR VARIANT_NAME_CS ILIKE '%MASK%'
  OR UPPER(VARIANT_NAME_CS) ILIKE '%ARM%';

UPDATE ORDER_PRODUCTS_0
SET COLOR = 'FIALOVÁ'
WHERE
  UPPER(VARIANT_NAME_CS) ILIKE '%FIAL%'
  OR VARIANT_NAME_CS ILIKE '%PURP%'
  OR UPPER(VARIANT_NAME_CS) ILIKE '%VIOL%'
  OR VARIANT_NAME_CS ILIKE '%MAG%'
  OR VARIANT_NAME_CS ILIKE '%VLT%'
  OR VARIANT_NAME_CS ILIKE '%LIL%'
  OR VARIANT_NAME_CS ILIKE '%FUCHS%';

UPDATE ORDER_PRODUCTS_0
SET COLOR = 'BÉŽOVÁ'
WHERE
  UPPER(VARIANT_NAME_CS) ILIKE '%BEZ%'
  OR VARIANT_NAME_CS ILIKE '%BÉŽ%'
  OR UPPER(VARIANT_NAME_CS) ILIKE '%BEI%';

UPDATE ORDER_PRODUCTS_0
SET COLOR = 'TYRKYSOVÁ'
WHERE
  UPPER(VARIANT_NAME_CS) ILIKE '%TYRK%'
  OR VARIANT_NAME_CS ILIKE '%TUR%'
  OR UPPER(VARIANT_NAME_CS) ILIKE '%TYR%'
  OR VARIANT_NAME_CS ILIKE '%TRQ%';

UPDATE ORDER_PRODUCTS_0
SET COLOR = 'VÍNOVÁ'
WHERE
  UPPER(VARIANT_NAME_CS) ILIKE '%VÍNO%'
  OR VARIANT_NAME_CS ILIKE '%VIN%'
  OR UPPER(VARIANT_NAME_CS) ILIKE '%WIN%';

UPDATE ORDER_PRODUCTS_0
SET COLOR = 'STŘÍBRNÁ'
WHERE
  UPPER(VARIANT_NAME_CS) ILIKE '%STR%'
  OR VARIANT_NAME_CS ILIKE '%STŘÍ%'
  OR UPPER(VARIANT_NAME_CS) ILIKE '%SILV%';

UPDATE ORDER_PRODUCTS_0
SET COLOR = 'KHAKI/OLIVE/PETROL'
WHERE
  UPPER(VARIANT_NAME_CS) ILIKE '%KHAKI%'
  OR VARIANT_NAME_CS ILIKE '%CAQ%'
  OR UPPER(VARIANT_NAME_CS) ILIKE '%OLIV%'
  OR VARIANT_NAME_CS ILIKE '%PETR%';

UPDATE ORDER_PRODUCTS_0
SET COLOR = 'ZLATÁ'
WHERE
  UPPER(VARIANT_NAME_CS) ILIKE '%ZLAT%'
  OR VARIANT_NAME_CS ILIKE '%GOLD%'
  OR UPPER(VARIANT_NAME_CS) ILIKE '%ZLA%';

UPDATE ORDER_PRODUCTS_0
SET COLOR = 'MIX BAREV'
WHERE
  UPPER(VARIANT_NAME_CS) ILIKE '%MIX%';

UPDATE ORDER_PRODUCTS_0
SET PRICE_CZK = '223'
WHERE PRICE_CZK = '';

ALTER TABLE ORDER_PRODUCTS_0 
ADD COLUMN VALUE_ITEM_ORDER INTEGER;

UPDATE ORDER_PRODUCTS_0
SET VALUE_ITEM_ORDER = CAST(PRICE_CZK AS INTEGER) * CAST(QUANTITY AS INTEGER);

# 2_block
UPDATE ORDER_PRODUCTS_0
SET CATEGORIES = 'Batohy, tašky, kufry'
WHERE
 UPPER(PRODUCT_NAME) ILIKE '%batoh%'
 OR PRODUCT_NAME ILIKE '%batohy%'
 OR UPPER(PRODUCT_NAME) ILIKE '%taška%'
 OR PRODUCT_NAME ILIKE '%taška%'
 AND categories = '';

UPDATE ORDER_PRODUCTS_0
SET CATEGORIES = 'Karimatky a matrace'
WHERE
 UPPER(PRODUCT_NAME) ILIKE '%karimat%'
 OR PRODUCT_NAME ILIKE '%karimat%'
 OR UPPER(PRODUCT_NAME) ILIKE '%matrac%'
 OR PRODUCT_NAME ILIKE '%matrac%'
 AND categories = '';

UPDATE ORDER_PRODUCTS_0
SET CATEGORIES = 'Vybavení'
WHERE
 PRODUCT_NAME ILIKE '%prut%'
 OR PRODUCT_NAME ILIKE '%popruh%'
 OR PRODUCT_NAME ILIKE '%položka%'
 OR PRODUCT_NAME ILIKE '%Slackline%'
 OR PRODUCT_NAME ILIKE '%Pouzdro%'
 OR PRODUCT_NAME ILIKE '%Přístavba%'
 OR PRODUCT_NAME ILIKE '%Podlážka%'
 OR PRODUCT_NAME ILIKE '%Kryt%'
 OR UPPER(PRODUCT_NAME) ILIKE '%baterka%'
 AND categories = '';

UPDATE ORDER_PRODUCTS_0
SET CATEGORIES = 'Stany'
WHERE CATEGORIES = 'Akce Coleman';

UPDATE ORDER_PRODUCTS_0
SET CATEGORIES = 'Unknown'
WHERE PRODUCT_NAME = 'Unknown';

# 3_block

CREATE OR REPLACE TABLE ORDER_ITEM AS
select 
ID
,ORDERS_ID
,PRODUCTS_ID
,PRODUCT_NAME
,VARIANT_NAME_CS
,PRICE_CZK
,QUANTITY
,VALUE_ITEM_ORDER
,WEIGHT
,CATEGORIES
,COLOR AS PRODUCT_COLOR
,PRODUCT_SIZE
from order_products_0
where dates between '2017-01-01 00:00:00' and '2023-01-01 00:00:00'

#### _5_4CAMPING_SHIPPING_METHODS

OUTPUT: SHIPPING_METHODS_TABLEA

In [ ]:
# 1_block
CREATE OR REPLACE TABLE SHIPPING_METHODS_TABLEAU AS (
SELECT
	"id" AS ID
	,"name_cs" AS SHIPPING_METHODS_NAME
	,"type" AS DELIVERY_TYPE
FROM SHIPPING_METHODS);

# 2_block
ALTER TABLE SHIPPING_METHODS_TABLEAU
ADD COLUMN CARRIER_NAME VARCHAR (20);

UPDATE SHIPPING_METHODS_TABLEAU
SET CARRIER_NAME = 
  CASE 
    WHEN SHIPPING_METHODS_NAME LIKE '%GLS%' THEN 'GLS' 
    WHEN SHIPPING_METHODS_NAME  LIKE '%DPD%' THEN 'DPD' 
    WHEN SHIPPING_METHODS_NAME  LIKE '%GEIS%' THEN 'GEIS' 
    WHEN SHIPPING_METHODS_NAME  LIKE '%PPL%' THEN 'PPL' 
    WHEN SHIPPING_METHODS_NAME  LIKE '%Balíkovna%' THEN 'BALIKOVNA' 
    WHEN SHIPPING_METHODS_NAME  LIKE '%iftago%' THEN 'LIFTAGO' 
    WHEN SHIPPING_METHODS_NAME  LIKE '%Česká%' THEN 'ČESKÁ POŠTA' 
    WHEN SHIPPING_METHODS_NAME  LIKE '%loženka%' THEN 'ULOŽENKA' 
    WHEN SHIPPING_METHODS_NAME  LIKE '%Dodo%' THEN 'DODO' 
    WHEN SHIPPING_METHODS_NAME  LIKE '%Packeta%' THEN 'PACKETA' 
    WHEN SHIPPING_METHODS_NAME  LIKE '%silkovna%' THEN 'PACKETA' 
    WHEN SHIPPING_METHODS_NAME  LIKE '%DHL%' THEN 'DHL'
    WHEN SHIPPING_METHODS_NAME  LIKE '%SPS%' THEN 'SPS - SK'
    WHEN SHIPPING_METHODS_NAME  LIKE '%venská po%' THEN 'SK pošta'
    WHEN SHIPPING_METHODS_NAME  LIKE '%test%' THEN 'test'
    ELSE 'OSOBNÍ ODBĚR' 
  END;

#### _6_4CAMPING_CATCHMEN_AREA

In [ ]:
--50.7703463	15.0611876

CREATE OR REPLACE TABLE CATCHMENT_AREA_HK AS (
-- Set the coordinates of the center point
WITH center_point_hk AS (
  SELECT 50.2113735 AS center_lat, 15.8185508 AS center_lon
),
-- Convert the radius from kilometers to degrees (approximately)
radius_deg AS (
  SELECT 7 / 111.045 AS radius_deg
),
-- Calculate the boundaries of the search area
boundaries AS (
  SELECT 
    center_lat - radius_deg AS min_lat,
    center_lat + radius_deg AS max_lat,
    center_lon - radius_deg AS min_lon,
    center_lon + radius_deg AS max_lon
  FROM center_point_hk, radius_deg
)
-- Query to retrieve records within the specified radius
SELECT ORDERS_GEO.LAT, ORDERS_GEO.LONG, ORDERS_GEO.ID, ORDERS_GEO.CITY, 'HRADEC KRÁLOVÉ' AS CATCHMENT_AREA, 'IN' AS IN_OUT_CA, DATE('2017-11-15') AS STORE_OPENING_DATE
FROM ORDERS_GEO, boundaries, center_point_hk, radius_deg
WHERE (ORDERS_GEO.LAT != '' AND ORDERS_GEO.LONG != '') AND ORDERS_GEO.LAT BETWEEN min_lat AND max_lat
  AND ORDERS_GEO.LONG BETWEEN min_lon AND max_lon
  AND (6371 * ACOS(
      COS(RADIANS(center_lat)) *
      COS(RADIANS(ORDERS_GEO.LAT)) *
      COS(RADIANS(ORDERS_GEO.LONG) - RADIANS(center_lon)) +
      SIN(RADIANS(center_lat)) *
      SIN(RADIANS(ORDERS_GEO.LAT))
    )) <= 20);

--LIBEREC

CREATE OR REPLACE TABLE CATCHMENT_AREA_LB AS (
-- Set the coordinates of the center point
WITH center_point AS (
  SELECT 50.7703463 AS center_lat, 15.0611876 AS center_lon
),
-- Convert the radius from kilometers to degrees (approximately)
radius_deg AS (
  SELECT 7 / 111.045 AS radius_deg
),
-- Calculate the boundaries of the search area
boundaries AS (
  SELECT 
    center_lat - radius_deg AS min_lat,
    center_lat + radius_deg AS max_lat,
    center_lon - radius_deg AS min_lon,
    center_lon + radius_deg AS max_lon
  FROM center_point, radius_deg
)
-- Query to retrieve records within the specified radius
SELECT ORDERS_GEO.LAT, ORDERS_GEO.LONG, ORDERS_GEO.ID, ORDERS_GEO.CITY, 'LIBEREC' AS CATCHMENT_AREA, 'IN' AS IN_OUT_CA, DATE('2020-3-19') AS STORE_OPENING_DATE
FROM ORDERS_GEO, boundaries, center_point, radius_deg
WHERE (ORDERS_GEO.LAT != '' AND ORDERS_GEO.LONG != '') AND ORDERS_GEO.LAT BETWEEN min_lat AND max_lat
  AND ORDERS_GEO.LONG BETWEEN min_lon AND max_lon
  AND (6371 * ACOS(
      COS(RADIANS(center_lat)) *
      COS(RADIANS(ORDERS_GEO.LAT)) *
      COS(RADIANS(ORDERS_GEO.LONG) - RADIANS(center_lon)) +
      SIN(RADIANS(center_lat)) *
      SIN(RADIANS(ORDERS_GEO.LAT))
    )) <= 20);

--OLOMOUC 49.596883	17.2628972

CREATE OR REPLACE TABLE CATCHMENT_AREA_OL AS (
-- Set the coordinates of the center point
WITH center_point AS (
  SELECT 49.596883 AS center_lat, 17.2628972 AS center_lon
),
-- Convert the radius from kilometers to degrees (approximately)
radius_deg AS (
  SELECT 7 / 111.045 AS radius_deg
),
-- Calculate the boundaries of the search area
boundaries AS (
  SELECT 
    center_lat - radius_deg AS min_lat,
    center_lat + radius_deg AS max_lat,
    center_lon - radius_deg AS min_lon,
    center_lon + radius_deg AS max_lon
  FROM center_point, radius_deg
)
-- Query to retrieve records within the specified radius
SELECT ORDERS_GEO.LAT, ORDERS_GEO.LONG, ORDERS_GEO.ID, ORDERS_GEO.CITY, 'OLOMOUC' AS CATCHMENT_AREA, 'IN' AS IN_OUT_CA, DATE('2021-4-16') AS STORE_OPENING_DATE
FROM ORDERS_GEO, boundaries, center_point, radius_deg
WHERE (ORDERS_GEO.LAT != '' AND ORDERS_GEO.LONG != '') AND ORDERS_GEO.LAT BETWEEN min_lat AND max_lat
  AND ORDERS_GEO.LONG BETWEEN min_lon AND max_lon
  AND (6371 * ACOS(
      COS(RADIANS(center_lat)) *
      COS(RADIANS(ORDERS_GEO.LAT)) *
      COS(RADIANS(ORDERS_GEO.LONG) - RADIANS(center_lon)) +
      SIN(RADIANS(center_lat)) *
      SIN(RADIANS(ORDERS_GEO.LAT))
    )) <= 20);

--CESKE BUDEJOVICE 48.9757711	14.481379

CREATE OR REPLACE TABLE CATCHMENT_AREA_CB AS (
-- Set the coordinates of the center point
WITH center_point AS (
  SELECT 48.9757711 AS center_lat, 14.481379 AS center_lon
),
-- Convert the radius from kilometers to degrees (approximately)
radius_deg AS (
  SELECT 7 / 111.045 AS radius_deg
),
-- Calculate the boundaries of the search area
boundaries AS (
  SELECT 
    center_lat - radius_deg AS min_lat,
    center_lat + radius_deg AS max_lat,
    center_lon - radius_deg AS min_lon,
    center_lon + radius_deg AS max_lon
  FROM center_point, radius_deg
)
-- Query to retrieve records within the specified radius
SELECT ORDERS_GEO.LAT, ORDERS_GEO.LONG, ORDERS_GEO.ID, ORDERS_GEO.CITY, 'ČESKÉ BUDĚJOVICE' AS CATCHMENT_AREA, 'IN' AS IN_OUT_CA, DATE('2021-07-05') AS STORE_OPENING_DATE
FROM ORDERS_GEO, boundaries, center_point, radius_deg
WHERE (ORDERS_GEO.LAT != '' AND ORDERS_GEO.LONG != '') AND ORDERS_GEO.LAT BETWEEN min_lat AND max_lat
  AND ORDERS_GEO.LONG BETWEEN min_lon AND max_lon
  AND (6371 * ACOS(
      COS(RADIANS(center_lat)) *
      COS(RADIANS(ORDERS_GEO.LAT)) *
      COS(RADIANS(ORDERS_GEO.LONG) - RADIANS(center_lon)) +
      SIN(RADIANS(center_lat)) *
      SIN(RADIANS(ORDERS_GEO.LAT))
    )) <= 20);

--BRNO - PALACKEHO 49.2212049	16.596644

CREATE OR REPLACE TABLE CATCHMENT_AREA_BRP AS (
-- Set the coordinates of the center point
WITH center_point AS (
  SELECT 49.2212049 AS center_lat, 16.596644 AS center_lon
),
-- Convert the radius from kilometers to degrees (approximately)
radius_deg AS (
  SELECT 7 / 111.045 AS radius_deg
),
-- Calculate the boundaries of the search area
boundaries AS (
  SELECT 
    center_lat - radius_deg AS min_lat,
    center_lat + radius_deg AS max_lat,
    center_lon - radius_deg AS min_lon,
    center_lon + radius_deg AS max_lon
  FROM center_point, radius_deg
)
-- Query to retrieve records within the specified radius
SELECT ORDERS_GEO.LAT, ORDERS_GEO.LONG, ORDERS_GEO.ID, ORDERS_GEO.CITY, 'BRNO - PAL' AS CATCHMENT_AREA, 'IN' AS IN_OUT_CA, DATE('2022-06-16') AS STORE_OPENING_DATE
FROM ORDERS_GEO, boundaries, center_point, radius_deg
WHERE (ORDERS_GEO.LAT != '' AND ORDERS_GEO.LONG != '') AND ORDERS_GEO.LAT BETWEEN min_lat AND max_lat
  AND ORDERS_GEO.LONG BETWEEN min_lon AND max_lon
  AND (6371 * ACOS(
      COS(RADIANS(center_lat)) *
      COS(RADIANS(ORDERS_GEO.LAT)) *
      COS(RADIANS(ORDERS_GEO.LONG) - RADIANS(center_lon)) +
      SIN(RADIANS(center_lat)) *
      SIN(RADIANS(ORDERS_GEO.LAT))
    )) <= 20);

--ZLIN - 49.233056, 17.666944

CREATE OR REPLACE TABLE OFF_CATCHMENT_AREA_ZL AS (
-- Set the coordinates of the center point
WITH center_point AS (
  SELECT 49.233056 AS center_lat, 17.666944 AS center_lon
),
-- Convert the radius from kilometers to degrees (approximately)
radius_deg AS (
  SELECT 7 / 111.045 AS radius_deg
),
-- Calculate the boundaries of the search area
boundaries AS (
  SELECT 
    center_lat - radius_deg AS min_lat,
    center_lat + radius_deg AS max_lat,
    center_lon - radius_deg AS min_lon,
    center_lon + radius_deg AS max_lon
  FROM center_point, radius_deg
)
-- Query to retrieve records within the specified radius
SELECT ORDERS_GEO.LAT, ORDERS_GEO.LONG, ORDERS_GEO.ID, ORDERS_GEO.CITY, 'ZLÍN' AS CATCHMENT_AREA, 'OUT' AS IN_OUT_CA, NULL AS STORE_OPENING_DATE
FROM ORDERS_GEO, boundaries, center_point, radius_deg
WHERE (ORDERS_GEO.LAT != '' AND ORDERS_GEO.LONG != '') AND ORDERS_GEO.LAT BETWEEN min_lat AND max_lat
  AND ORDERS_GEO.LONG BETWEEN min_lon AND max_lon
  AND (6371 * ACOS(
      COS(RADIANS(center_lat)) *
      COS(RADIANS(ORDERS_GEO.LAT)) *
      COS(RADIANS(ORDERS_GEO.LONG) - RADIANS(center_lon)) +
      SIN(RADIANS(center_lat)) *
      SIN(RADIANS(ORDERS_GEO.LAT))
    )) <= 20);

--KARLOVY VARY - 50.230556, 12.8725

CREATE OR REPLACE TABLE OFF_CATCHMENT_AREA_KV AS (
-- Set the coordinates of the center point
WITH center_point AS (
  SELECT 50.230556 AS center_lat, 12.8725 AS center_lon
),
-- Convert the radius from kilometers to degrees (approximately)
radius_deg AS (
  SELECT 7 / 111.045 AS radius_deg
),
-- Calculate the boundaries of the search area
boundaries AS (
  SELECT 
    center_lat - radius_deg AS min_lat,
    center_lat + radius_deg AS max_lat,
    center_lon - radius_deg AS min_lon,
    center_lon + radius_deg AS max_lon
  FROM center_point, radius_deg
)
-- Query to retrieve records within the specified radius
SELECT ORDERS_GEO.LAT, ORDERS_GEO.LONG, ORDERS_GEO.ID, ORDERS_GEO.CITY, 'KARLOVY VARY' AS CATCHMENT_AREA, 'OUT' AS IN_OUT_CA, NULL AS STORE_OPENING_DATE
FROM ORDERS_GEO, boundaries, center_point, radius_deg
WHERE (ORDERS_GEO.LAT != '' AND ORDERS_GEO.LONG != '') AND ORDERS_GEO.LAT BETWEEN min_lat AND max_lat
  AND ORDERS_GEO.LONG BETWEEN min_lon AND max_lon
  AND (6371 * ACOS(
      COS(RADIANS(center_lat)) *
      COS(RADIANS(ORDERS_GEO.LAT)) *
      COS(RADIANS(ORDERS_GEO.LONG) - RADIANS(center_lon)) +
      SIN(RADIANS(center_lat)) *
      SIN(RADIANS(ORDERS_GEO.LAT))
    )) <= 20);

--USTI NAD LABEM - 50.659167, 14.041667

CREATE OR REPLACE TABLE OFF_CATCHMENT_AREA_UL AS (
-- Set the coordinates of the center point
WITH center_point AS (
  SELECT 50.659167 AS center_lat, 14.041667 AS center_lon
),
-- Convert the radius from kilometers to degrees (approximately)
radius_deg AS (
  SELECT 7 / 111.045 AS radius_deg
),
-- Calculate the boundaries of the search area
boundaries AS (
  SELECT 
    center_lat - radius_deg AS min_lat,
    center_lat + radius_deg AS max_lat,
    center_lon - radius_deg AS min_lon,
    center_lon + radius_deg AS max_lon
  FROM center_point, radius_deg
)
-- Query to retrieve records within the specified radius
SELECT ORDERS_GEO.LAT, ORDERS_GEO.LONG, ORDERS_GEO.ID, ORDERS_GEO.CITY, 'USTI NAD LABEM' AS CATCHMENT_AREA, 'OUT' AS IN_OUT_CA, NULL AS STORE_OPENING_DATE
FROM ORDERS_GEO, boundaries, center_point, radius_deg
WHERE (ORDERS_GEO.LAT != '' AND ORDERS_GEO.LONG != '') AND ORDERS_GEO.LAT BETWEEN min_lat AND max_lat
  AND ORDERS_GEO.LONG BETWEEN min_lon AND max_lon
  AND (6371 * ACOS(
      COS(RADIANS(center_lat)) *
      COS(RADIANS(ORDERS_GEO.LAT)) *
      COS(RADIANS(ORDERS_GEO.LONG) - RADIANS(center_lon)) +
      SIN(RADIANS(center_lat)) *
      SIN(RADIANS(ORDERS_GEO.LAT))
    )) <= 20);

--JIHLAVA 49.400278, 15.590556

CREATE OR REPLACE TABLE OFF_CATCHMENT_AREA_JI AS (
-- Set the coordinates of the center point
WITH center_point AS (
  SELECT 49.400278 AS center_lat, 15.590556 AS center_lon
),
-- Convert the radius from kilometers to degrees (approximately)
radius_deg AS (
  SELECT 7 / 111.045 AS radius_deg
),
-- Calculate the boundaries of the search area
boundaries AS (
  SELECT 
    center_lat - radius_deg AS min_lat,
    center_lat + radius_deg AS max_lat,
    center_lon - radius_deg AS min_lon,
    center_lon + radius_deg AS max_lon
  FROM center_point, radius_deg
)
-- Query to retrieve records within the specified radius
SELECT ORDERS_GEO.LAT, ORDERS_GEO.LONG, ORDERS_GEO.ID, ORDERS_GEO.CITY, 'JIHLAVA' AS CATCHMENT_AREA, 'OUT' AS IN_OUT_CA, NULL AS STORE_OPENING_DATE
FROM ORDERS_GEO, boundaries, center_point, radius_deg
WHERE (ORDERS_GEO.LAT != '' AND ORDERS_GEO.LONG != '') AND ORDERS_GEO.LAT BETWEEN min_lat AND max_lat
  AND ORDERS_GEO.LONG BETWEEN min_lon AND max_lon
  AND (6371 * ACOS(
      COS(RADIANS(center_lat)) *
      COS(RADIANS(ORDERS_GEO.LAT)) *
      COS(RADIANS(ORDERS_GEO.LONG) - RADIANS(center_lon)) +
      SIN(RADIANS(center_lat)) *
      SIN(RADIANS(ORDERS_GEO.LAT))
    )) <= 20);

SELECT LAT, COUNT(LAT) AS VYSKYT
FROM OFF_CATCHMENT_AREA_JI
GROUP BY LAT
ORDER BY VYSKYT DESC

SELECT *
FROM OFF_CATCHMENT_AREA_JI

# 2_block
--UNIONIEM VYTVPRENI TABULKU

CREATE OR REPLACE TABLE CATCHMENT_AREA AS(
SELECT *
FROM CATCHMENT_AREA_BRP

UNION ALL

SELECT *
FROM CATCHMENT_AREA_CB

UNION ALL

SELECT *
FROM CATCHMENT_AREA_HK

UNION ALL

SELECT *
FROM CATCHMENT_AREA_LB

UNION ALL

SELECT *
FROM CATCHMENT_AREA_OL

UNION ALL

SELECT *
FROM OFF_CATCHMENT_AREA_JI

UNION ALL

SELECT *
FROM OFF_CATCHMENT_AREA_KV

UNION ALL

SELECT *
FROM OFF_CATCHMENT_AREA_UL

UNION ALL

SELECT *
FROM OFF_CATCHMENT_AREA_ZL)

#### _7_4ACAMPING_TABLEAU

PUVODNI PLAN BYL NAHRAT TYTO TABULKY ZVLAST DO TABELAU. ALE DOSTALI JSME INFO, ŽE IDEALNĚ MIT 1, MAX 2 TABULKY.
PROTO JSME JOINOVAL:
 - CATCHMENT_AREA	
 - CUSTOMER_TABLEAU	
 - GEOLOCATION_TABLEAU	
 - ORDER_ITEM	
 - ORDER_TABLEAU	
 - ORDERS_GEO	
 - SHIPPING_METHODS_TABLEAU

In [ ]:
CREATE OR REPLACE TABLE TABLEAU AS
SELECT
 OT.ID
,OT.IDATE
,OT.ID_CUSTOMER
,CT."pohlavi"
,SMT.DELIVERY_TYPE
,SMT.SHIPPING_METHODS_NAME
,OT.PAYMENT_METHODS_NAME
,IT.PRODUCT_NAME
,IT.PRICE_CZK
,IT.QUANTITY
,IT.VALUE_ITEM_ORDER
,IT.PRODUCT_COLOR
,IT.PRODUCT_SIZE
,IT.WEIGHT
,IT.CATEGORIES
,GT.REGION
,GT.SUBREGION
,GT.CITY
,GT.NEIGHBORHOOD
,GT.ZIP_CODE
,GT.LONG
,GT.LAT
,CA.IN_OUT_CA
,CA.CATCHMENT_AREA
,CA.STORE_OPENING_DATE
FROM ORDER_TABLEAU OT 
LEFT JOIN CUSTOMER_TABLEAU CT ON CT.ID_CUSTOMER = OT.ID_CUSTOMER --OPRAVENO OT.ID NA OT.ID_CUSTOMER
LEFT JOIN SHIPPING_METHODS_TABLEAU SMT ON SMT.ID = OT.SHIPPING_METHODS_ID
LEFT JOIN ORDER_ITEM IT ON IT.ORDERS_ID = OT.ID
LEFT JOIN GEOLOCATION_TABLEAU GT ON GT.ID = OT.ID 
LEFT JOIN CATCHMENT_AREA CA ON CA.ID = OT.ID;

#### POSLEDNI: PŘES COMPONENT NAHRAT DO TABLEA

- CALENDAR
        - tabulka kalendar, statnima svatkami

- POCET_OBYVATEL 
        - tabulka kde mame, obec, okres, kraj, pocet obivatele a vek
- TABLEAU
        - vytvorena jedna velka tabulka s nasima datama